# Panel data regression model
For the empirical part of my thesis project for master of Business Economics, I have used Panel data regression model with. I used linearmodels library. First, the data is normalized to be prepared for use in regresion model. Then the model is fitted.

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels import stats
from linearmodels import PanelOLS
from linearmodels import RandomEffects
from statsmodels.regression.mixed_linear_model import MixedLM
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Import data from Excel file
Data = pd.read_csv('Full_data.csv', na_values=np.nan, index_col=[0])
Data = Data.set_index(['Country', 'Bank'])
Data.head()

Year  Innovat_%Total  Innovat_cluster  Risk_%Total  \
Country Bank                                                                   
Bahrain Bank ABC Islamic  2010        0.090909              0.0      0.09375   
        Bank ABC Islamic  2011        0.072727              0.0      0.06250   
        Bank ABC Islamic  2012        0.109091              0.0      0.09375   
        Bank ABC Islamic  2013        0.072727              0.0      0.03125   
        Bank ABC Islamic  2014        0.072727              0.0      0.06250   

                          Risk_cluster  BInnov_clus  BRisk_clus  BankID  \
Country Bank                                                              
Bahrain Bank ABC Islamic           1.0          1.0         0.0      21   
        Bank ABC Islamic           1.0          1.0         0.0      21   
        Bank ABC Islamic           0.0          1.0         0.0      21   
        Bank ABC Islamic           1.0          1.0         0.0      21   
        Bank ABC Islamic           1.0          1.0         0.0      21   

                               ROA       ROE  ...  NPL_ratio  Window  \
Country Bank                                  ...                      
Bahrain Bank ABC Islamic  0.001671  0.003548  ...   0.165544     4.0   
        Bank ABC Islamic  0.007875  0.013454  ...   0.176510     4.0   
        Bank ABC Islamic  0.007759  0.013188  ...   0.105877     4.0   
        Bank ABC Islamic  0.012196  0.018490  ...   0.064275     3.0   
        Bank ABC Islamic  0.011350  0.021500  ...   0.035650     3.0   

                          Full_fledged  Total_IB  GDP_growth  \
Country Bank                                                   
Bahrain Bank ABC Islamic           7.0      11.0    0.043343   
        Bank ABC Islamic           7.0      11.0    0.019835   
        Bank ABC Islamic           7.0      11.0    0.037281   
        Bank ABC Islamic           7.0      10.0    0.054168   
        Bank ABC Islamic           7.0      10.0    0.043504   

                          GDP_percapita_growth  Inflation  Real_interest_rate  \
Country Bank                                                                    
Bahrain Bank ABC Islamic             -0.003561   0.019619           -0.001802   
        Bank ABC Islamic             -0.009919  -0.004015           -0.026845   
        Bank ABC Islamic              0.019892   0.027583            0.029318   
        Bank ABC Islamic              0.042075   0.033010            0.055296   
        Bank ABC Islamic              0.027067   0.026463            0.076673   

                               FDI_net    FDI_inflow  
Country Bank                                          
Bahrain Bank ABC Islamic  1.782715e+08  1.557710e+08  
        Bank ABC Islamic  1.127660e+08  7.808511e+08  
        Bank ABC Islamic -1.028723e+09  1.544681e+09  
        Bank ABC Islamic -3.196064e+09  3.727713e+09  
        Bank ABC Islamic -1.913032e+09  1.518617e+09  

[5 rows x 22 columns]

In [3]:
cols = ['Innovat_%Total', 'Risk_%Total', 'ROA', 'ROE', 'firm_age', 'AssetSize_USD', 'NPL_ratio', 'Window', 'Full_fledged',
        'Total_IB', 'GDP_growth', 'GDP_percapita_growth', 'Inflation', 'Real_interest_rate', 'FDI_net', 'FDI_inflow']
Data[cols].groupby(by = 'Country').describe().transpose()

Country                    Bahrain    Bangladesh        Brunei         Egypt  \
Innovat_%Total count  6.400000e+01  6.500000e+01  8.000000e+00  1.000000e+01   
               mean   1.511364e-01  1.879720e-01  2.068182e-01  1.600000e-01   
               std    5.419561e-02  4.537230e-02  5.227978e-02  2.944232e-02   
               min    7.272727e-02  5.454545e-02  1.272727e-01  1.090909e-01   
               25%    1.090909e-01  1.454545e-01  1.727273e-01  1.454545e-01   
...                            ...           ...           ...           ...   
FDI_inflow     min    6.489362e+07  9.012866e+08 -1.505508e+08 -4.827000e+08   
               25%    1.557710e+08  1.584403e+09  3.732568e+08  4.297200e+09   
               50%    5.188830e+08  2.071560e+09  5.162026e+08  6.818400e+09   
               75%    1.518617e+09  2.602962e+09  6.911703e+08  8.106800e+09   
               max    3.727713e+09  2.940222e+09  8.649055e+08  9.010100e+09   

Country                  Indonesia        Jordan         Kenya        Kuwait  \
Innovat_%Total count  1.600000e+01  1.900000e+01  0.000000e+00  4.300000e+01   
               mean   1.909091e-01  9.473684e-02           NaN  1.581395e-01   
               std    4.146092e-02  6.286077e-02           NaN  6.991701e-02   
               min    1.272727e-01  3.636364e-02           NaN  3.636364e-02   
               25%    1.590909e-01  3.636364e-02           NaN  9.090909e-02   
...                            ...           ...           ...           ...   
FDI_inflow     min    4.541714e+09  9.498592e+08  1.780646e+08 -2.137757e+07   
               25%    1.890983e+10  1.548451e+09  6.326655e+08  2.846476e+08   
               50%    2.053762e+10  1.600282e+09  8.209376e+08  4.858067e+08   
               75%    2.328174e+10  1.946597e+09  1.380174e+09  1.433630e+09   
               max    2.512073e+10  2.178451e+09  1.625921e+09  3.259068e+09   

Country                   Malaysia          Oman      Pakistan         Qatar  \
Innovat_%Total count  3.200000e+01  1.300000e+01  4.100000e+01  3.800000e+01   
               mean   2.409091e-01  2.489510e-01  1.977827e-01  1.421053e-01   
               std    5.985854e-02  3.268062e-02  5.209577e-02  4.181610e-02   
               min    1.090909e-01  1.818182e-01  3.636364e-02  5.454545e-02   
               25%    2.181818e-01  2.363636e-01  1.818182e-01  1.090909e-01   
...                            ...           ...           ...           ...   
FDI_inflow     min    8.570094e+09 -2.172432e+09  8.590000e+08 -2.812637e+09   
               25%    9.368470e+09  1.286346e+09  1.673000e+09 -8.403846e+08   
               50%    1.061943e+10  1.938855e+09  2.022000e+09  9.385165e+08   
               75%    1.129628e+10  2.917295e+09  2.496000e+09  1.070879e+09   
               max    1.511944e+10  6.342263e+09  5.590000e+09  8.124736e+09   

Country               Saudi Arabia         Sudan        Turkey           UAE  
Innovat_%Total count  2.500000e+01  1.100000e+01  3.800000e+01  2.600000e+01  
               mean   1.360000e-01  1.471074e-01  1.669856e-01  1.552448e-01  
               std    4.927016e-02  3.196542e-02  4.589083e-02  6.768537e-02  
               min    5.454545e-02  9.090909e-02  5.454545e-02  1.818182e-02  
               25%    1.090909e-01  1.272727e-01  1.454545e-01  1.136364e-01  
...                            ...           ...           ...           ...  
FDI_inflow     min    1.418844e+09  1.063768e+09  1.082000e+09  7.152096e+09  
               25%    4.562037e+09  1.135787e+09  9.565000e+09  8.796770e+09  
               50%    8.141027e+09  1.653120e+09  1.333700e+10  9.604773e+09  
               75%    1.218237e+10  1.728373e+09  1.392900e+10  1.035422e+10  
               max    3.645767e+10  2.311461e+09  2.204700e+10  1.107154e+10  

[128 rows x 16 columns]

In [5]:
# Normalization
norm_cols = ['firm_age', 'AssetSize_USD', 'Total_IB', 'FDI_net', 'FDI_inflow']
Bank_max = Data[norm_cols].groupby(by=['Country', 'Bank']).max()
Bank_min = Data[norm_cols].groupby(by=['Country', 'Bank']).min()
Data_merged = pd.merge(Data, Bank_max, left_index=True, right_index=True, suffixes=('','_max'))
Data_merged = pd.merge(Data_merged, Bank_min, left_index=True, right_index=True, suffixes=('','_min'))

for col in norm_cols:
    Data_merged[col] = (Data_merged[col]-Data_merged[col+'_min'])/(Data_merged[col+'_max']-Data_merged[col+'_min'])
    
Data_merged.head()

Year  Innovat_%Total  Innovat_cluster  Risk_%Total  \
Country Bank                                                                 
Bahrain Al Baraka Bank  2010        0.254545              1.0      0.15625   
        Al Baraka Bank  2013        0.254545              1.0      0.09375   
        Al Baraka Bank  2014        0.218182              1.0      0.12500   
        Al Baraka Bank  2015        0.218182              1.0      0.12500   
        Al Baraka Bank  2016        0.290909              1.0      0.09375   

                        Risk_cluster  BInnov_clus  BRisk_clus  BankID  \
Country Bank                                                            
Bahrain Al Baraka Bank           0.0          0.0         1.0       6   
        Al Baraka Bank           0.0          0.0         1.0       6   
        Al Baraka Bank           0.0          0.0         1.0       6   
        Al Baraka Bank           0.0          0.0         1.0       6   
        Al Baraka Bank           1.0          0.0         1.0       6   

                             ROA       ROE  ...  firm_age_max  \
Country Bank                                ...                 
Bahrain Al Baraka Bank  0.003357  0.024513  ...          35.0   
        Al Baraka Bank  0.000563  0.005555  ...          35.0   
        Al Baraka Bank  0.001132  0.012195  ...          35.0   
        Al Baraka Bank  0.003298  0.035064  ...          35.0   
        Al Baraka Bank  0.000988  0.011371  ...          35.0   

                        AssetSize_USD_max  Total_IB_max   FDI_net_max  \
Country Bank                                                            
Bahrain Al Baraka Bank       2.385250e+09          11.0  3.126064e+09   
        Al Baraka Bank       2.385250e+09          11.0  3.126064e+09   
        Al Baraka Bank       2.385250e+09          11.0  3.126064e+09   
        Al Baraka Bank       2.385250e+09          11.0  3.126064e+09   
        Al Baraka Bank       2.385250e+09          11.0  3.126064e+09   

                        FDI_inflow_max  firm_age_min  AssetSize_USD_min  \
Country Bank                                                              
Bahrain Al Baraka Bank    3.727713e+09          26.0       1.347000e+09   
        Al Baraka Bank    3.727713e+09          26.0       1.347000e+09   
        Al Baraka Bank    3.727713e+09          26.0       1.347000e+09   
        Al Baraka Bank    3.727713e+09          26.0       1.347000e+09   
        Al Baraka Bank    3.727713e+09          26.0       1.347000e+09   

                        Total_IB_min   FDI_net_min  FDI_inflow_min  
Country Bank                                                        
Bahrain Al Baraka Bank          10.0 -3.196064e+09    6.489362e+07  
        Al Baraka Bank          10.0 -3.196064e+09    6.489362e+07  
        Al Baraka Bank          10.0 -3.196064e+09    6.489362e+07  
        Al Baraka Bank          10.0 -3.196064e+09    6.489362e+07  
        Al Baraka Bank          10.0 -3.196064e+09    6.489362e+07  

[5 rows x 32 columns]

## Panel data model

In [8]:
cols = ['Year', 'Innovat_%Total', 'Innovat_cluster', 'BInnov_clus', 'Risk_%Total', 'Risk_cluster', 'BRisk_clus', 'BankID',
        'ROA', 'ROE', 'firm_age', 'AssetSize_USD', 'NPL_ratio', 'Window', 'Full_fledged', 'Total_IB', 'GDP_growth',
        'GDP_percapita_growth', 'Inflation', 'Real_interest_rate', 'FDI_net', 'FDI_inflow']

Data_model = Data_merged[cols].reset_index().set_index(['Bank', 'Year'])
year = Data_model.index.get_level_values(1)
Data_model.head(10)

Country  Innovat_%Total  Innovat_cluster  BInnov_clus  \
Bank           Year                                                          
Al Baraka Bank 2010  Bahrain        0.254545              1.0          0.0   
               2013  Bahrain        0.254545              1.0          0.0   
               2014  Bahrain        0.218182              1.0          0.0   
               2015  Bahrain        0.218182              1.0          0.0   
               2016  Bahrain        0.290909              1.0          0.0   
               2017  Bahrain        0.236364              1.0          0.0   
               2018  Bahrain        0.218182              1.0          0.0   
               2019  Bahrain        0.236364              1.0          0.0   
               2011  Bahrain             NaN              NaN          NaN   
               2012  Bahrain             NaN              NaN          NaN   

                     Risk_%Total  Risk_cluster  BRisk_clus  BankID       ROA  \
Bank           Year                                                            
Al Baraka Bank 2010      0.15625           0.0         1.0       6  0.003357   
               2013      0.09375           0.0         1.0       6  0.000563   
               2014      0.12500           0.0         1.0       6  0.001132   
               2015      0.12500           0.0         1.0       6  0.003298   
               2016      0.09375           1.0         1.0       6  0.000988   
               2017      0.09375           0.0         1.0       6 -0.002316   
               2018      0.12500           0.0         1.0       6 -0.008416   
               2019      0.12500           0.0         1.0       6  0.002846   
               2011          NaN           NaN         NaN       6  0.001624   
               2012          NaN           NaN         NaN       6 -0.007500   

                          ROE  ...  NPL_ratio  Window  Full_fledged  Total_IB  \
Bank           Year            ...                                              
Al Baraka Bank 2010  0.024513  ...        NaN     4.0           7.0       1.0   
               2013  0.005555  ...   0.456746     3.0           7.0       0.0   
               2014  0.012195  ...   0.116887     3.0           7.0       0.0   
               2015  0.035064  ...   0.061375     3.0           7.0       0.0   
               2016  0.011371  ...   0.068891     3.0           7.0       0.0   
               2017 -0.027676  ...   0.200971     3.0           7.0       0.0   
               2018 -0.074970  ...   0.228339     3.0           7.0       0.0   
               2019  0.023655  ...   0.105832     NaN           NaN       NaN   
               2011  0.014474  ...   0.545042     4.0           7.0       1.0   
               2012 -0.062868  ...   0.539625     4.0           7.0       1.0   

                     GDP_growth  GDP_percapita_growth  Inflation  \
Bank           Year                                                
Al Baraka Bank 2010    0.043343             -0.003561   0.019619   
               2013    0.054168              0.042075   0.033010   
               2014    0.043504              0.027067   0.026463   
               2015    0.028630              0.001805   0.018395   
               2016    0.034735             -0.004411   0.028006   
               2017    0.038117             -0.009328   0.013873   
               2018    0.019572             -0.029388   0.020848   
               2019    0.018196             -0.026308        NaN   
               2011    0.019835             -0.009919  -0.004015   
               2012    0.037281              0.019892   0.027583   

                     Real_interest_rate   FDI_net  FDI_inflow  
Bank           Year                                            
Al Baraka Bank 2010           -0.001802  0.533734    0.024811  
               2013            0.055296  0.000000    1.000000  
               2014            0.076673  0.202943    0.396886  
             

In [9]:
# model 1
exog_vars1 = ['Innovat_%Total', 'firm_age', 'AssetSize_USD', 'NPL_ratio', 'Total_IB',
             'GDP_percapita_growth', 'Inflation', 'Real_interest_rate', 'FDI_inflow']
exog1 = Data_model[exog_vars1]

mod1 = RandomEffects(Data_model['ROA'], exog1)
re_res1 = mod1.fit()
print(re_res1)

C:\Users\Amin\Miniconda3\lib\site-packages\linearmodels\utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                        RandomEffects Estimation Summary                        
Dep. Variable:                    ROA   R-squared:                        0.2132
Estimator:              RandomEffects   R-squared (Between):              0.6440
No. Observations:                 237   R-squared (Within):              -0.0546
Date:                Fri, Aug 21 2020   R-squared (Overall):              0.5146
Time:                        19:57:20   Log-likelihood                    816.32
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      6.8660
Entities:                          31   P-value                           0.0000
Avg Obs:                       7.6452   Distribution:                   F(9,228)
Min Obs:                       2.0000                                           
Max Obs:                       13.000   F-statistic (robust):             6.8660
                            

In [10]:
# Wald test 1
restriction = np.identity(9)
value = np.zeros((1,9))
re_res1.wald_test(restriction=restriction, value=value)

Linear Equality Hypothesis Test
H0: Linear equality constraint is valid
Statistic: 61.7936
P-value: 0.0000
Distributed: chi2(9)
WaldTestStatistic, id: 0x1dbe5ec8

In [11]:
# model 2
exog_vars2 = ['Innovat_%Total', 'Risk_%Total', 'firm_age', 'AssetSize_USD', 'NPL_ratio', 'Total_IB',
             'GDP_percapita_growth', 'Inflation', 'Real_interest_rate', 'FDI_inflow']
exog2 = Data_model[exog_vars2]

mod2 = RandomEffects(Data_model['ROA'], exog2)
re_res2 = mod2.fit()
print(re_res2)

                        RandomEffects Estimation Summary                        
Dep. Variable:                    ROA   R-squared:                        0.2184
Estimator:              RandomEffects   R-squared (Between):              0.6430
No. Observations:                 237   R-squared (Within):              -0.0407
Date:                Fri, Aug 21 2020   R-squared (Overall):              0.5180
Time:                        19:58:46   Log-likelihood                    818.00
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      6.3446
Entities:                          31   P-value                           0.0000
Avg Obs:                       7.6452   Distribution:                  F(10,227)
Min Obs:                       2.0000                                           
Max Obs:                       13.000   F-statistic (robust):             6.3446
                            